Name: Mehmetcan TÜTÜNCÜ

I hereby declare that I observed the honour code of the university when preparing the homework.

## Programming Homework 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [2]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [3]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'t'})$"))
for c,p in zip(alphabet,T[letter2idx['t']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

('a', '0.0393295')
('b', '0.0001590')
('c', '0.0037195')
('d', '0.0000674')
('e', '0.0892434')
('f', '0.0009218')
('g', '0.0000404')
('h', '0.3352928')
('i', '0.0666758')
('j', '0.0000054')
('k', '0.0000162')
('l', '0.0146273')
('m', '0.0009110')
('n', '0.0011051')
('o', '0.0913053')
('p', '0.0000809')
('q', '0.0000027')
('r', '0.0310281')
('s', '0.0245378')
('t', '0.0171177')
('u', '0.0185732')
('v', '0.0000027')
('w', '0.0078702')
('x', '0.0000000')
('y', '0.0121422')
('z', '0.0002776')
('.', '0.2449470')


#### Part 1
For the Markov(1) model, $p( x_1,x_2,...,x_N | x_0 ) = p(x_1|x_0)p(x_2|x_1) ... p(x_n|x_{n-1})$

We can utilize numpy.random.choice function to generate variates from discrete distributions.

(Probability vectors are normalized to sum up 1 in this example since they have precisional errors causing sums of 1.00001 or 0.99999)


In [4]:
import numpy as np
tempMatrix= np.asarray(T)
probMatrix= np.zeros(shape=(27,27))
for letter in alphabet:
    probMatrix[letter2idx[letter]] = (tempMatrix[letter2idx[letter]].astype(np.float))/(sum(tempMatrix[letter2idx[letter]].astype(np.float)))
#probMatrix[letter2idx['q']]   #-> vector of probabilities, Pr(X_(i+1)|X_i='q')
#print np.sum(probMatrix[letter2idx['a']])
#needed to normalize so they sum up to 1. Some of them are 1.0001 or 0.9999
def guessNextLetter(previousLetter):
    return np.random.choice(alphabet,p= probMatrix[letter2idx[previousLetter]])
print guessNextLetter('t')    
   
def genRandomString(N):
    randomString = '.'
    randomString+= guessNextLetter(randomString[-1])
    for i in range(1,N):
        randomString+= guessNextLetter(randomString[-1])
    return randomString
#generates 10 random strings
for i in range(2,12):
    print(genRandomString(i)[1:])




s
t.
tow
myir
h.cot
bacly.
it.itib
es.tone.
ge.ot.far
.gmy.ichra
susosplyiom


### Part 2
For a missing letter in the string, only variables that matter for the distribution is the  first known letter that comes before and the first letter that comes after the missing one under Markov(1) assumption.
Define a substring $X_{0:(N+1)}$ with the length of $N+2$ where $x_0$ and $x_{N+1}$ are known and $x_{0:N}$ are missing.
$$p(x_{1:N}{\mid}x_0=\hat{x}_0,x_{N+1}=\hat{x}_{N+1}) \propto p(x_{N+1}=\hat{x}_{N+1}{\mid}x_{1:N},x_0=\hat{x}_0)  p(x_N{\mid}x_{1:N-1},x_0=\hat{x}_0)p(x_{N-1}{\mid}x_{1:N-2},x_0=\hat{x}_0)...p(x_1{\mid}x_0=\hat{x}_0)$$
equivalently,
$$p(x_{1:N}{\mid}x_0=\hat{x}_0,x_{N+1}=\hat{x}_{N+1}) \propto p(x_{N+1}=\hat{x}_{N+1}{\mid}x_N) p(x_N{\mid}x_{N-1})p(x_{N-1}{\mid}x_{N-2})...p(x_2|x_1)p(x_1|x_0=\hat{x}_0)$$

$\begin{equation}\label{eq:1}
p(x_{1}{\mid}x_0=\hat{x}_0,x_{N+1}=\hat{x}_{N+1}) \propto p(x_N=\hat{x}_N \mid x_1) p(x_1 \mid x_0=\hat{x}_0)
\end{equation}$
In the expression above, $p(x_N=\hat{x}_N \mid x_1)$ is a column in $N$'th power of the transition matrix( $T^N$).
For the solution strategy, program will iterate over the string, find the longest consecutive missing streak at the beginning of the string. After finding the longest possible missing substring, it will change a character according to the distribution, 

In [5]:
def probBetweenLetters(letterFirst,letterLast,countMissing=1):
    xFirst= letter2idx[letterFirst]
    xLast= letter2idx[letterLast]
    #calculation of the probability vector
    probabilityVector=np.linalg.matrix_power(probMatrix,countMissing)[:,xLast] * probMatrix[xFirst]
    #normalization of the vector
    probabilityVector /= sum(probabilityVector)
    return probabilityVector
def fillMissing(string):
    string= '.'+string+'.'
    
    missingIndices = [i for i, x in enumerate(list(string)) if x in ['_','?']]
    
    if not missingIndices:
        return string[1:][:-1]
    
    #detects the first consecutive missing streak and randoms the first letter
    firstMissingIndex=missingIndices[0]
    i=0
    if(len(missingIndices) > 1):
        while( (missingIndices[i+1]-missingIndices[i])==1 ):
            i=i+1
            if((i+1)==len(missingIndices)):
                break    
        
        lastMissingIndex = firstMissingIndex+ i
        guess= np.random.choice(alphabet,p=probBetweenLetters(string[firstMissingIndex-1],string[lastMissingIndex+1],
                                             lastMissingIndex-firstMissingIndex+1))
    else:
        guess=np.random.choice(alphabet,p=probBetweenLetters(string[firstMissingIndex-1],string[firstMissingIndex+1],1))
        
    #remove dot's from the beginning and the end,  replace predicted letter with the first missing value detected
    changedString=string[1:firstMissingIndex]+guess+ string[(firstMissingIndex+1):-1]
    
    #recursive solution on each recursion, we change simply one letter 
    return(fillMissing(changedString))
    
for string in test_strings:
    print string
    for i in range(10):
        print "Trial ",i,": ",fillMissing(string)

th__br__n.f_x.
Trial  0 :  the.brsin.fix.
Trial  1 :  the.brgen.fex.
Trial  2 :  the.br.an.fex.
Trial  3 :  the.bruan.fex.
Trial  4 :  the.br.hn.fax.
Trial  5 :  the.bri.n.fux.
Trial  6 :  the.brean.fex.
Trial  7 :  tha.br.un.fex.
Trial  8 :  the.brorn.fex.
Trial  9 :  the.br.in.fex.
_u_st__n_.to_be._nsw_r__
Trial  0 :  ourstcono.to.be.answarle
Trial  1 :  bussteind.tombe.inswhri.
Trial  2 :  bunsthind.to.be.answorou
Trial  3 :  tursthand.toybe.answarer
Trial  4 :  cutsthong.to.be.inswir.y
Trial  5 :  buesthend.toube.inswbrud
Trial  6 :  wurst.ine.to.be.inswor.s
Trial  7 :  mutsth.nd.toube.answeres
Trial  8 :  burstoong.torbe.answer.t
Trial  9 :  tuisthend.to.be.onswired
i__at_._a_h_n_._e_r_i_g
Trial  0 :  illato.vathon..he.r.irg
Trial  1 :  ithato.fathine.pe.rhing
Trial  2 :  ithati.ta.hens.tear.ing
Trial  3 :  is.ath.fashend.me.r.ing
Trial  4 :  ingate.cathund.herrsing
Trial  5 :  igsatw.wathind.wearking
Trial  6 :  i.hato..athind.bepruing
Trial  7 :  incate.tachend.berrging
Trial  8

### Part 3
A similar idea is used to evaluate best possible canditates. Evaluating argmax at every step for each $x_i$, will guarantee the highest probability possible. Since $p(x_i)$ here are all positive and selected iteratively $max(\prod{x_i}) = \prod{max(p_i}$) 

In [39]:
def BestFillMissing(string,logProbability=0):
    string= '.'+string+'.'
    
    missingIndices = [i for i, x in enumerate(list(string)) if x in ['_','?']]
    
    if not missingIndices:
        return (string[1:][:-1],logProbability)
    
    #detects the first consecutive missing streak and randoms the first letter
    firstMissingIndex=missingIndices[0]
    i=0
    if(len(missingIndices) > 1):
        while( (missingIndices[i+1]-missingIndices[i])==1 ):
            i=i+1
            if((i+1)==len(missingIndices)):
                break    
        
        lastMissingIndex = firstMissingIndex+ i
        probVector= probBetweenLetters(string[firstMissingIndex-1],string[lastMissingIndex+1],
                                             lastMissingIndex-firstMissingIndex+1)
        guess=np.argmax(probVector) #instead of generating a random variate, we return the maximum element of probVector
        logProbabilityNew= logProbability+  np.log(probVector[guess])
        guess=letter2idx.keys()[letter2idx.values().index(guess)]
    else:
        probVector = probBetweenLetters(string[firstMissingIndex-1],string[firstMissingIndex+1],1)
        guess=np.argmax(probVector)
        logProbabilityNew= logProbability+  np.log(probVector[guess])
        guess=letter2idx.keys()[letter2idx.values().index(guess)]
        
    #remove dot's from the beginning and the end,  replace predicted letter with the first missing value detected
    changedString=string[1:firstMissingIndex]+guess+ string[(firstMissingIndex+1):-1]
    #recursive solution on each recursion, we change simply one letter, and accumulate the logProbability value
    return(BestFillMissing(changedString,logProbabilityNew))    

for string in test_strings:
    filledString,probability= BestFillMissing(string)
    print "Missing String:",string, "Log Probability"
    print "Filled String :",filledString, probability

Missing String: th__br__n.f_x. Log Probability
Filled String : the.br.an.fex. -3.07433488138
Missing String: _u_st__n_.to_be._nsw_r__ Log Probability
Filled String : oursthend.to.be.answered -11.0693283596
Missing String: i__at_._a_h_n_._e_r_i_g Log Probability
Filled String : in.ath.wathend.he.r.ing -11.6360900332
Missing String: q___t.___z._____t.__.___.__. Log Probability
Filled String : qur.t.thiz.the.at.an.the.an. -22.9236427638


### Part 4
This version of model can only detect relationships within the distance of 1. Deeper Markov models will perform better than current model. Instead of feeding letters one by one , we could also predict the words that are completely missing with a holistic approach using a pre-created English dictionary to make predictions by Markov(1) model with a big transition matrix storing $(N_{Words})^2$ probabilities (it might exceed memory limitations if there is any).